In [1]:
import jax.numpy as jnp
import jax.random as jrd
from jax import jit, vmap, grad

In [19]:
rng = jrd.PRNGKey(seed=0)

In [20]:
jrd.uniform(rng, (2,)), jrd.uniform(rng, (2,))

(Array([0.21629536, 0.8041241 ], dtype=float32),
 Array([0.21629536, 0.8041241 ], dtype=float32))

In [21]:
new_rng, sub_rng = jrd.split(rng, 2)
rng, new_rng, sub_rng

(Array([0, 0], dtype=uint32),
 Array([4146024105,  967050713], dtype=uint32),
 Array([2718843009, 1272950319], dtype=uint32))

In [23]:
rng = jrd.PRNGKey(seed=0)
for i in range(10):
    rng, rng_task = jrd.split(rng, 2)
    # do stuff ...
    _ = jrd.uniform(rng_task, shape=(1000,))


In [31]:
import jax.numpy as jnp

array = jnp.ones((10,))
array

Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [34]:
array = array.at[2].set(0.)

In [35]:
array

Array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [47]:
def L2(x, y, k = None):
    diff = x - y
    return jnp.sqrt(jnp.sum(jnp.square(diff)))

In [48]:
# jit_L2 = jit(L2, static_argnames=["k"])
jit_L2 = jit(L2)

jit_L2(jnp.array([1, 6, 7]), jnp.array([8, -3, -4]), 5)

Array(15.842979, dtype=float32)

In [56]:
@jit
def g(x, y, k):
    kk = jnp.full_like(x, k)
    return jnp.log(x + kk * y)
    # return jnp.log(x + jnp.ones((k,)) * y)


In [57]:
g(
    jnp.ones((10, )),
    jnp.arange(0, 10),
    10
)

Array([0.       , 2.3978953, 3.0445225, 3.4339871, 3.713572 , 3.9318256,
       4.1108737, 4.26268  , 4.394449 , 4.5108595], dtype=float32)

In [29]:
import jax.numpy as jnp
import jax.random as jrd
from jax import jit, vmap


@jit
def L2(x, y):
    diff = x - y
    return jnp.sqrt(jnp.sum(jnp.square(diff)))

vmaped_L2 = jit(vmap(L2, in_axes=(0, 0)))


vmaped_L2(
    jnp.arange(0, 10),
    jnp.arange(10, 20) + jrd.normal(jrd.PRNGKey(0), (10,))
)


Array([ 9.627889, 10.264231,  9.817472,  9.263181,  9.559696,  9.847856,
        9.328647,  9.409136, 10.731689, 10.567303], dtype=float32)

In [33]:
import jax.numpy as jnp
import jax.random as jrd
from jax import jit, vmap, grad


def loss_fn(w, b, xs, ys):
    # Root mean squared error
    pred = w * xs + b
    return jnp.sqrt(jnp.sum(jnp.square(pred - ys)))


grad_fn = grad(loss_fn, argnums=(0, 1))
grad_fn(13., 0., 42.0, 500.0)

(Array(42., dtype=float32, weak_type=True),
 Array(1., dtype=float32, weak_type=True))

In [34]:
from jax.tree_util import register_pytree_node_class

from functools import partial

In [52]:
@register_pytree_node_class
class F:
    """This definition of the class, makes it reactive to internal change.
    If self.operator is modified, the class is recompiled using the correct new function
    """
    def __init__(self, op) -> None:
        self.operator = op

    @jit
    def apply(self, *args):
        return self.operator(*args)
    
    def tree_flatten(self):
        children = ()
        aux_data = {"operator": self.operator}
        return (children, aux_data)

    @classmethod
    def tree_unflatten(cls, aux_data, children):
        return cls(aux_data["operator"])

In [53]:
f = F(jnp.exp)

In [54]:
f.apply(4.)

Array(54.59815, dtype=float32, weak_type=True)

In [55]:
f.operator = jnp.log
f.apply(4.)

Array(1.3862944, dtype=float32, weak_type=True)

In [50]:
f.operator(4.)

Array(1.3862944, dtype=float32, weak_type=True)

In [51]:
f.apply(4.)

Array(54.59815, dtype=float32, weak_type=True)